In [ ]:
import pandas as pd
import numpy as np
import zipfile,nltk
from nltk.corpus import PlaintextCorpusReader 
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re
from nltk.corpus import stopwords
import heapq as hq
import sys
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import csv
import os
import string
import re
from bs4 import BeautifulSoup
from ast import literal_eval
nltk.download('stopwords')
from collections import Counter



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# nltk.download('stopwords')

# stopli = set(stopwords.words('english'))
# # stopli.add('\x1a')
# print(stopli)
stopwords = set(stopwords.words('english'))

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/IR_Project_Shared/Location/FinalFilteredData.csv",header=None,names=['Place','Location','City','Rating','Raw_Review'])
data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: x.strip('b'))
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: x.replace("\n", " ")) #replace newlines with spaces
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: x.replace("\t", " ")) #replace tab indents with spaces
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: x.replace('\\r',' ').replace('\\n',' ').replace('\\x1a',' ').replace('\\t',' ')) #replace "\\r, \\n, \\x1a" by spaces
data['Raw_Review']= data['Raw_Review'].apply(lambda x: re.sub(r"http\S+", "", str(x)))  #remove URLs
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: BeautifulSoup(x, "lxml").get_text()) #remove tags
data['Raw_Review']= data['Raw_Review'].apply(lambda x: re.sub("\S*\d\S*", "", str(x)).strip()) #remove words with numbers
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: re.sub(r"n\'t", " not", str(x))) #replace n't with not
data['Raw_Review']= data['Raw_Review'].apply(lambda x: re.sub(r"\'re", " are", str(x))) #replace 're with are
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: re.sub(r"\'s", " is", str(x))) #replace 's' with is
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: re.sub(r"\'ll", " will", str(x))) #replace 'll with will
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: re.sub(r"\'d", " would", str(x))) #replace 'd' with would
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: re.sub(r"\'ve", " have", str(x))) #replace 've with have
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: re.sub(r"\'m", " am", str(x))) #replace 'm with am
data['Raw_Review'] = data['Raw_Review'].apply(lambda x: re.sub("[^A-Za-z0-9]+", " ", str(x))) #remove punctuation

data['Raw_Review'] = data['Raw_Review'].apply(lambda x: ' '.join(worD.lower() for worD in x.split() if worD.lower() not in stopwords)) # remove stopwords

data['Raw_Review']= data['Raw_Review'].apply(lambda x: word_tokenize(x)) #word tokenization

In [ ]:
data

In [ ]:
data_place_loc=data[['Place','Location','City']]
data_place_loc.drop_duplicates(['Place'],inplace=True)
data_place_loc.reset_index(drop=True,inplace=True)
data_place_loc

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Place,Location,City
0,3Cs Mall,"[39.7113703, 140.1179655469886]",New Delhi
1,Aai Museum,"[-12.779714250000001, 15.739036595404063]",Mumbai
2,Aarey Bhaskar Garden,"[19.1715961, 72.8636339]",Mumbai
3,Aarey Colony,"[19.1503904, 72.87426783008218]",Mumbai
4,Adventure Island,"[-17.14634975, -144.3481775735695]",New Delhi
...,...,...,...
338,Worli Fort,"[19.02374855, 72.81683615091026]",Mumbai
339,Worli Sea Face,"[19.005448, 72.8137566]",Mumbai
340,Yamuna Sports Complex,"[28.6635715, 77.31182552880594]",New Delhi
341,Yogmaya Temple,"[28.5257019, 77.1822181]",New Delhi


In [ ]:
import numpy as np
data_tokens_review= data.groupby('Place').agg({'Raw_Review': lambda x: list(x),'Rating':lambda x:np.mean(x)}).reset_index()



In [ ]:
data_tokens_review

,Place,Raw_Review,Rating
0,3Cs Mall,"[[ordered, idli, south, indian, filter, coffee...",4.000000
1,Aai Museum,"[[art, lovers, nice, place, museum, four, sect...",5.000000
2,Aarey Bhaskar Garden,"[[well, maintained, park, goregaon, east, suff...",5.000000
3,Aarey Colony,"[[place, film, shootings, quite, deserted, pla...",3.545455
4,Adventure Island,"[[may, amusement, parks, experience, horrible,...",4.507246
...,...,...,...
338,Worli Fort,"[[lovely, place, visit, open, view, terrace, f...",5.000000
339,Worli Sea Face,"[[place, sea, facing, view, worli, bandra, sea...",4.418182
340,Yamuna Sports Complex,"[[member, complex, since, joined, feel, great,...",3.750000
341,Yogmaya Temple,"[[temple, dedicated, yogamaya, considered, bab...",2.000000


In [ ]:
# data_final
data_result=pd.merge(data_place_loc,data_tokens_review, on="Place")
data_result
data_result.to_csv("/content/drive/MyDrive/IR_Project_Shared/Location/finaldataUpdated_merged.csv",index=False)

In [ ]:
data_read_final=pd.read_csv("/content/drive/MyDrive/IR_Project_Shared/Location/finaldataUpdated_merged.csv")

In [ ]:
data_read_final[data_read_final['Place']=='Bandra-Worli Sea Link']

,Place,Location,City,Raw_Review,Rating
26,Bandra-Worli Sea Link,"[19.03594, 72.8171302]",Mumbai,"[['must', 'see', 'mumbai', 'looks', 'beautiful', 'night', 'great', 'place', 'get', 'view', 'wide', 'open', 'ocean'], ['toll', 'road', 'built', 'water', 'gets', 'southern', 'mumbai', 'northern', 'part', 'city', 'traffic'], ['metro', 'completed', 'best', 'way', 'get', 'heart', 'mumbai', 'bandra', 'view', 'arabian', 'sea', 'good', 'air', 'quality', 'allows'], ['bandra', 'sea', 'link', 'heart', 'm...",4.286754


In [ ]:
data_read_final['Raw_Review']= data_read_final['Raw_Review'].apply(literal_eval)
data_read_final

,Place,Location,City,Raw_Review,Rating
0,3Cs Mall,"[39.7113703, 140.1179655469886]",New Delhi,"[[ordered, idli, south, indian, filter, coffee...",4.000000
1,Aai Museum,"[-12.779714250000001, 15.739036595404063]",Mumbai,"[[art, lovers, nice, place, museum, four, sect...",5.000000
2,Aarey Bhaskar Garden,"[19.1715961, 72.8636339]",Mumbai,"[[well, maintained, park, goregaon, east, suff...",5.000000
3,Aarey Colony,"[19.1503904, 72.87426783008218]",Mumbai,"[[place, film, shootings, quite, deserted, pla...",3.545455
4,Adventure Island,"[-17.14634975, -144.3481775735695]",New Delhi,"[[may, amusement, parks, experience, horrible,...",4.507246
...,...,...,...,...,...
338,Worli Fort,"[19.02374855, 72.81683615091026]",Mumbai,"[[lovely, place, visit, open, view, terrace, f...",5.000000
339,Worli Sea Face,"[19.005448, 72.8137566]",Mumbai,"[[place, sea, facing, view, worli, bandra, sea...",4.418182
340,Yamuna Sports Complex,"[28.6635715, 77.31182552880594]",New Delhi,"[[member, complex, since, joined, feel, great,...",3.750000
341,Yogmaya Temple,"[28.5257019, 77.1822181]",New Delhi,"[[temple, dedicated, yogamaya, considered, bab...",2.000000


In [ ]:
def x(list_z):
  d = []
  for elem in list_z:
    for elem_k in elem:
      d.append(elem_k)

  return list(set(d))

In [ ]:
data_read_final['Raw_Review']=data_read_final['Raw_Review'].apply(x)
data_read_final.to_csv("/content/drive/MyDrive/IR_Project_Shared/Project_data_mid.csv",index=False)


In [ ]:
data_read_final

,Place,Location,City,Raw_Review,Rating
0,3Cs Mall,"[39.7113703, 140.1179655469886]",New Delhi,"[better, ordered, gut, pre, strong, see, made,...",4.000000
1,Aai Museum,"[-12.779714250000001, 15.739036595404063]",Mumbai,"[lovers, two, brave, nice, pilaji, sections, t...",5.000000
2,Aarey Bhaskar Garden,"[19.1715961, 72.8636339]",Mumbai,"[making, actually, east, lively, maintained, p...",5.000000
3,Aarey Colony,"[19.1503904, 72.87426783008218]",Mumbai,"[soothing, mumbai, left, others, calm, bustle,...",3.545455
4,Adventure Island,"[-17.14634975, -144.3481775735695]",New Delhi,"[clean, budget, foul, comments, aeroplane, wri...",4.507246
...,...,...,...,...,...
338,Worli Fort,"[19.02374855, 72.81683615091026]",Mumbai,"[clearly, arabian, terrace, link, bay, worli, ...",5.000000
339,Worli Sea Face,"[19.005448, 72.8137566]",Mumbai,"[clean, mumbai, aroma, residence, believe, bit...",4.418182
340,Yamuna Sports Complex,"[28.6635715, 77.31182552880594]",New Delhi,"[clean, management, possible, feel, completely...",3.750000
341,Yogmaya Temple,"[28.5257019, 77.1822181]",New Delhi,"[killed, prison, krishna, fell, girl, dedicate...",2.000000


In [ ]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]
